In [1]:
%pip install langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
import key

# 1. LLM 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3, openai_api_key=key.key["OPEN_API_KEY"])

# 2. 질문 생성 프롬프트
question_prompt = ChatPromptTemplate.from_template("""
다음 문서를 기반으로 적절한 질문을 하나 생성하세요. 답은 반드시 문서 안에 있어야 합니다.

문서:
{context}

질문:
""")

# 3. 답변 생성 프롬프트
answer_prompt = ChatPromptTemplate.from_template("""
질문에 대해 아래 문서를 참고해서 최대한 정확하게 답변하세요. 문서에 명시되지 않은 내용은 답하지 마세요.

문서:
{context}

질문:
{question}

답변:
""")


In [3]:
# #S1

# import fitz  # PyMuPDF
# import re
# import json

# # PDF 열기 및 목차 이후 본문 텍스트 추출 (5페이지부터)
# pdf_path = r"D:\TECHLAB\2차프로젝트_Chatbot\data\raw\IFRS_S1.pdf"
# doc = fitz.open(pdf_path)

# start_page = 4  # 0-indexed → 5페이지부터
# full_text = "\n".join(doc[i].get_text() for i in range(start_page, len(doc)))

# # 섹션 정의 (start ~ end 기준, 정규표현식)
# section_defs = [
#     ("chunk_01_objective", r"\nObjective\n", r"\nScope\n"),
#     ("chunk_02_scope", r"\nScope\n", r"\nConceptual foundations\n"),
#     ("chunk_03_concepts", r"\nConceptual foundations\n", r"\nCore content\n"),  # ⬅️ 'Fair presentation'~'Connected information' 포함
#     ("chunk_04_governance", r"\nGovernance\n", r"\nStrategy\n"),
#     ("chunk_05_strategy", r"\nStrategy\n", r"\nRisk management\n"),
#     ("chunk_06_risk", r"\nRisk management\n", r"\nMetrics and targets\n"),
#     ("chunk_07_metrics", r"\nMetrics and targets\n", r"\nGeneral requirements\n"),  # ← 전에는 Sources of guidance였음
#     ("chunk_08_general", r"\nGeneral requirements\n", r"\nJudgements\n"),
#     ("chunk_09_judgements", r"\nJudgements\n", r"\nMeasurement uncertainty\n"),
#     ("chunk_10_uncertainty", r"\nMeasurement uncertainty\n", r"\nErrors\n"),
#     ("chunk_11_errors", r"\nErrors\n", r"Appendix A\nDefined terms"),
#     ("chunk_12_appendix_a", r"Appendix A\nDefined terms", r"Appendix B\nApplication guidance"),
#     ("chunk_13_appendix_b", r"Appendix B\nApplication guidance", r"Appendix C\nSources of guidance"),
#     ("chunk_14_appendix_c", r"Appendix C\nSources of guidance", r"Appendix D\nQualitative characteristics"),
#     ("chunk_15_appendix_d", r"Appendix D\nQualitative characteristics", r"Appendix E\nEffective date and transition"),
#     ("chunk_16_appendix_e", r"Appendix E\nEffective date and transition", r"Disclosure of Sustainability-related Financial Information issued"),
#     ("chunk_17_approval", r"Disclosure of Sustainability-related Financial Information issued", None)
# ]


# # 1차 청크 생성
# section_chunks = []
# for chunk_id, start_pattern, end_pattern in section_defs: 
#     start_match = re.search(start_pattern, full_text, flags=re.IGNORECASE)
#     if not start_match:
#         continue
#     start_pos = start_match.start()
#     end_pos = len(full_text)
#     if end_pattern:
#         end_match = re.search(end_pattern, full_text[start_pos + 1:], flags=re.IGNORECASE)
#         if end_match:
#             end_pos = start_pos + 1 + end_match.start()
#     section_text = full_text[start_pos:end_pos].strip()
#     section_chunks.append({"id": chunk_id, "text": section_text})

# # 확인
# print(f"✅ 1차 섹션 수: {len(section_chunks)}")
# print(section_chunks[0]["id"], ":", section_chunks[0]["text"][:200], "...")


# documents = [
#     Document(page_content=chunk['text'], metadata={"chunk_id": chunk['id'], "source": "IFRS_S1"})
#     for chunk in section_chunks
# ]

# # 📌 5. 2차 청크 생성 (1000자 기준)
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.schema import Document

# def split_into_chunks(documents, max_chars=1000):
#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=max_chars,
#         chunk_overlap=20,
#         separators=["\n\n", "\n", ".", " ", ""]
#     )

#     new_docs = []
#     for doc in documents:
#         chunks = splitter.split_text(doc.page_content)
#         for i, chunk in enumerate(chunks):
#             new_docs.append(Document(
#                 page_content=chunk,
#                 metadata={
#                     "chunk_id": f"{doc.metadata['chunk_id']}_{i+1:02}",
#                     "source": doc.metadata["chunk_id"]
#                 }
#             ))
    
#     return new_docs

# # ✅ dict → Document로 변환한 리스트
# documents2 = [
#     Document(page_content=chunk['text'], metadata={"chunk_id": chunk['id'], "source": "IFRS_S1"})
#     for chunk in section_chunks
# ]

# # ✅ 2차 청크 생성
# second_level_docs = split_into_chunks(documents2)  # ✅ 여기 section_chunks ❌ → documents ✅
# print(f"✅ 2차 청크 수: {len(second_level_docs)}")

In [4]:
# qa_results = []

# for doc in second_level_docs :  
#     context = doc.page_content
#     chunk_id = doc.metadata["chunk_id"]
#     source = doc.metadata["source"]

#     # 1. 질문 생성
#     question = llm.invoke(question_prompt.format(context=context)).content.strip()

#     # 2. 답변 생성
#     answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

#     # 3. 저장
#     qa_results.append({
#         "chunk_id": chunk_id,
#         "source": source,
#         "question": question,
#         "answer": answer,
#         "context": context[:300] + "..."  # 긴 context는 요약 출력
#     })


# print(len(qa_results))
# print(qa_results)

In [5]:
# #저장하기 json
# import json

# # 파일로 저장
# with open("qa_results.json", "w", encoding="utf-8") as f:
#     json.dump(qa_results, f, ensure_ascii=False, indent=2)

# print("✅ qa_results.json 파일 저장 완료")


#저장하기 vector
# from langchain.schema import Document

# qa_documents = []

# for item in qa_results:
#     qa_documents.append(Document(
#         page_content = f"Q: {item['question']}\nA: {item['answer']}",
#         metadata = {
#             "question": item["question"],
#             "chunk_id": item["chunk_id"],
#             "source": item["source"]
#         }
#     ))


# from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
# import key

# # 임베딩 모델 설정
# embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# # 벡터스토어로 저장
# qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
# qa_vectorstore.save_local("vectorstoresqa/qa_results_openai")

# print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/qa_results_openai")



In [6]:
# #평가하기
# from langchain.evaluation import load_evaluator
# from langchain_openai import ChatOpenAI
# import key

# # ✅ 평가에 사용할 LLM
# llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])




# # ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
# evaluator = load_evaluator("context_qa", llm=llm)

# # ✅ 평가 실행
# evaluated_results = []

# for item in qa_results:
#     result = evaluator.evaluate_strings(
#         input={"question": item["question"], "context": item["context"]},
#         prediction=item["answer"],
#         reference=item["answer"]  # ⬅️ 추가됨
#     )
#     evaluated_results.append({
#         "chunk_id": item["chunk_id"],
#         "question": item["question"],
#         "answer": item["answer"],
#         "context": item["context"],
#         "score": result["score"],
#         "reasoning": result.get("reasoning", "")
#     })


# # 결과 출력(상위 5개)
# import pandas as pd

# df_eval = pd.DataFrame(evaluated_results)
# display(df_eval[["chunk_id", "score", "question", "answer"]].head())


# #점수 구체화
# evaluator = load_evaluator(
#     evaluator="score_string",
#     criteria={"relevance": "질문과 답변의 관련성"},
#     normalize_by=10,
#     llm=llm
# )

# # DataFrame으로 변환
# df_eval = pd.DataFrame(evaluated_results)

# # CSV로 저장
# df_eval.to_csv("qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

# print("✅ LLM 평가 결과 저장 완료: qa_llm_eval_results.csv")



In [7]:
# # 코사인 유사도 검사
# from langchain_openai import OpenAIEmbeddings
# import key

# embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


# import numpy as np
# from numpy.linalg import norm

# def cosine_similarity(vec1, vec2):
#     return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


# evaluated_results = []

# for item in qa_results:
#     try:
#         answer_emb = embedding_model.embed_query(item["answer"])
#         context_emb = embedding_model.embed_query(item["context"])
#         score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
#     except Exception as e:
#         print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
#         score = None
    
#     evaluated_results.append({
#         "chunk_id": item["chunk_id"],
#         "question": item["question"],
#         "answer": item["answer"],
#         "score_cosine": score
#     })

# import pandas as pd

# df_eval = pd.DataFrame(evaluated_results)
# df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

# display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


# import pandas as pd

# # 1. DataFrame으로 변환
# df = pd.DataFrame(evaluated_results)

# # 2. CSV로 저장
# df.to_csv("qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

# print("✅ CSV 파일 저장 완료: qa_results_cosine_eval.csv")



In [8]:
#사용예시

# retriever = qa_vectorstore.as_retriever(k=3)
# query = "기후 관련 리스크의 종류는?"
# retrieved_docs = retriever.invoke(query)

# for doc in retrieved_docs:
#     print(f"\n📌 질문: {doc.metadata['question']}")
#     print(f"🔎 출처: {doc.metadata['chunk_id']}")
#     print(f"📝 내용:\n{doc.page_content}")


In [9]:
# #S2

# import fitz  # PyMuPDF
# import re
# import json

# # PDF 열기 및 목차 이후 본문 텍스트 추출 (5페이지부터)
# pdf_path = r"D:\TECHLAB\2차프로젝트_Chatbot\data\raw\IFRS_S2.pdf"
# doc = fitz.open(pdf_path)

# start_page = 4  # 0-indexed → 5페이지부터
# full_text = "\n".join(doc[i].get_text() for i in range(start_page, len(doc)))

# # 섹션 정의 (start ~ end 기준, 정규표현식)
# section_defs = [
#     ("chunk_01_objective", r"\nObjective\n", r"\nScope\n"),
#     ("chunk_02_scope", r"\nScope\n", r"\nGovernance\n"),
#     ("chunk_03_governance", r"\nGovernance\n", r"\nStrategy\n"),
#     ("chunk_04_strategy", r"\nStrategy\n", r"\nRisk management\n"),
#     ("chunk_05_risk", r"\nRisk management\n", r"\nMetrics and targets\n"),
#     ("chunk_06_metrics", r"\nMetrics and targets\n", r"Appendix A\nDefined terms"),
#     ("chunk_07_appendix_a", r"Appendix A\nDefined terms", r"Appendix B\nApplication guidance"),
#     ("chunk_08_appendix_b", r"Appendix B\nApplication guidance", r"Appendix C\nEffective date and transition"),
#     ("chunk_09_appendix_c", r"Appendix C\nEffective date and transition", r"Approval by the ISSB"),
#     ("chunk_10_approval", r"Approval by the ISSB", None),
# ]

# # 1차 청크 생성
# section_chunks = []
# for chunk_id, start_pattern, end_pattern in section_defs: 
#     start_match = re.search(start_pattern, full_text, flags=re.IGNORECASE)
#     if not start_match:
#         continue
#     start_pos = start_match.start()
#     end_pos = len(full_text)
#     if end_pattern:
#         end_match = re.search(end_pattern, full_text[start_pos + 1:], flags=re.IGNORECASE)
#         if end_match:
#             end_pos = start_pos + 1 + end_match.start()
#     section_text = full_text[start_pos:end_pos].strip()
#     section_chunks.append({"id": chunk_id, "text": section_text})

# # 확인
# print(f"✅ 1차 섹션 수: {len(section_chunks)}")
# print(section_chunks[0]["id"], ":", section_chunks[0]["text"][:200], "...")


# # ✅ dict → Document 변환
# documents = [
#     Document(page_content=chunk['text'], metadata={"chunk_id": chunk['id'], "source": "IFRS_S2"})
#     for chunk in section_chunks
# ]

# # ✅ 2차 청크 분할 함수
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.schema import Document

# def split_into_chunks(documents, max_chars=1000):
#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=max_chars,
#         chunk_overlap=20,
#         separators=["\n\n", "\n", ".", " ", ""]
#     )

#     new_docs = []
#     for doc in documents:
#         chunks = splitter.split_text(doc.page_content)
#         for i, chunk in enumerate(chunks):
#             new_docs.append(Document(
#                 page_content=chunk,
#                 metadata={
#                     "chunk_id": f"{doc.metadata['chunk_id']}_{i+1:02}",
#                     "source": doc.metadata["chunk_id"]
#                 }
#             ))
    
#     return new_docs

# # ✅ 2차 청크 생성 및 저장
# second_level_docs = split_into_chunks(documents)
# print(f"✅ 2차 청크 수: {len(second_level_docs)}")

In [10]:
# qa_results = []

# for doc in second_level_docs :  
#     context = doc.page_content
#     chunk_id = doc.metadata["chunk_id"]
#     source = doc.metadata["source"]

#     # 1. 질문 생성
#     question = llm.invoke(question_prompt.format(context=context)).content.strip()

#     # 2. 답변 생성
#     answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

#     # 3. 저장
#     qa_results.append({
#         "chunk_id": chunk_id,
#         "source": source,
#         "question": question,
#         "answer": answer,
#         "context": context[:300] + "..."  # 긴 context는 요약 출력
#     })


# print(len(qa_results))
# print(qa_results)

In [11]:
# #저장하기 json
# import json

# # 파일로 저장
# with open("IFRS_02_qa_results.json", "w", encoding="utf-8") as f:
#     json.dump(qa_results, f, ensure_ascii=False, indent=2)

# print("✅ qa_results.json 파일 저장 완료")


# # 저장하기 vector
# from langchain.schema import Document

# qa_documents = []

# for item in qa_results:
#     qa_documents.append(Document(
#         page_content = f"Q: {item['question']}\nA: {item['answer']}",
#         metadata = {
#             "question": item["question"],
#             "chunk_id": item["chunk_id"],
#             "source": item["source"]
#         }
#     ))


# from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
# import key

# # 임베딩 모델 설정
# embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# # 벡터스토어로 저장
# qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
# qa_vectorstore.save_local("vectorstoresqa/IFRS_02_qa_results_openai")

# print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/IFRS_02_qa_results_openai")



In [12]:
# #평가하기
# from langchain.evaluation import load_evaluator
# from langchain_openai import ChatOpenAI
# import key

# # ✅ 평가에 사용할 LLM
# llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])




# # ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
# evaluator = load_evaluator("context_qa", llm=llm)

# # ✅ 평가 실행
# evaluated_results = []

# for item in qa_results:
#     result = evaluator.evaluate_strings(
#         input={"question": item["question"], "context": item["context"]},
#         prediction=item["answer"],
#         reference=item["answer"]  # ⬅️ 추가됨
#     )
#     evaluated_results.append({
#         "chunk_id": item["chunk_id"],
#         "question": item["question"],
#         "answer": item["answer"],
#         "context": item["context"],
#         "score": result["score"],
#         "reasoning": result.get("reasoning", "")
#     })


# # 결과 출력(상위 5개)
# import pandas as pd

# df_eval = pd.DataFrame(evaluated_results)
# display(df_eval[["chunk_id", "score", "question", "answer"]].head())


# #점수 구체화
# evaluator = load_evaluator(
#     evaluator="score_string",
#     criteria={"relevance": "질문과 답변의 관련성"},
#     normalize_by=10,
#     llm=llm
# )

# # DataFrame으로 변환
# df_eval = pd.DataFrame(evaluated_results)

# # CSV로 저장
# df_eval.to_csv("IFRS_02_qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

# print("✅ LLM 평가 결과 저장 완료: IFRS_02_qa_llm_eval_results.csv")



In [13]:
# # 코사인 유사도 검사
# from langchain_openai import OpenAIEmbeddings
# import key

# embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


# import numpy as np
# from numpy.linalg import norm

# def cosine_similarity(vec1, vec2):
#     return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


# evaluated_results = []

# for item in qa_results:
#     try:
#         answer_emb = embedding_model.embed_query(item["answer"])
#         context_emb = embedding_model.embed_query(item["context"])
#         score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
#     except Exception as e:
#         print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
#         score = None
    
#     evaluated_results.append({
#         "chunk_id": item["chunk_id"],
#         "question": item["question"],
#         "answer": item["answer"],
#         "score_cosine": score
#     })

# import pandas as pd

# df_eval = pd.DataFrame(evaluated_results)
# df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

# display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


# import pandas as pd

# # 1. DataFrame으로 변환
# df = pd.DataFrame(evaluated_results)

# # 2. CSV로 저장
# df.to_csv("IFRS_02_qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

# print("✅ CSV 파일 저장 완료: IFRS_02_qa_results_cosine_eval.csv")



In [14]:
# # GRI
# import fitz
# import re
# from langchain.schema import Document

# # 1. PDF 열기
# pdf_path = r"D:\TECHLAB\2차프로젝트_Chatbot\data\raw\Consolidated Set of the GRI Standards.pdf"
# doc = fitz.open(pdf_path)

# # 2. GRI 전체 문서 페이지 정의 (0-index 기준)
# section_page_defs = [
#     ("GRI_1", 4, 40),
#     ("GRI_2", 40, 94),
#     ("GRI_3", 94, 121),
#     ("GRI_11", 121, 292),
#     ("GRI_13", 292, 383),
#     ("GRI_14", 383, 478),
#     ("GRI_101", 478, 488),
#     ("GRI_102", 488, 519),
#     ("GRI_103", 519, 540),
#     ("GRI_201", 540, 547),
#     ("GRI_202", 547, 550),
#     ("GRI_203", 550, 558),
#     ("GRI_204", 558, 566),
#     ("GRI_205", 566, 569),
#     ("GRI_206", 569, 574),
#     ("GRI_207", 574, 588),
#     ("GRI_301", 588, 663),
#     ("GRI_302", 663, 669),
#     ("GRI_303", 669, 691),
#     ("GRI_305", 691, 703),
#     ("GRI_306", 703, 741),
#     ("GRI_308", 741, 754),
#     ("GRI_401", 754, 764),
#     ("GRI_402", 764, 777),
#     ("GRI_403", 777, 794),
#     ("GRI_404", 794, 800),
#     ("GRI_405", 800, 807),
#     ("GRI_406", 807, 827),
#     ("GRI_407", 827, 837),
#     ("GRI_408", 837, 848),
#     ("GRI_409", 848, 858),
#     ("GRI_410", 858, 866),
#     ("GRI_411", 866, 876),
#     ("GRI_412", 876, 888),
#     ("GRI_414", 888, 895),
#     ("GRI_415", 895, 898),
#     ("GRI_416", 898, 903),
#     ("GRI_417", 903, 926),
#     ("GRI_418", 926, 936),
#     ("GRI_Glossary", 936, len(doc)),
# ]

# # 3. 헤더 패턴 정의 (Introduction, 1., 2., ..., Glossary, Bibliography, Appendix)
# header_pattern = re.compile(
#     r"(?P<header>Introduction|Glossary|Bibliography|Appendix[\s\S]*?|^\d+\..+)",
#     flags=re.IGNORECASE | re.MULTILINE
# )

# # 4. 각 문서 내 세부 청크 분할
# all_documents = []

# for gri_id, start_page, end_page in section_page_defs:
#     text = "\n".join(doc[i].get_text() for i in range(start_page, end_page))

#     matches = list(header_pattern.finditer(text))
#     if not matches:
#         print(f"⚠️ 헤더 없음: {gri_id}")
#         continue

#     for i, match in enumerate(matches):
#         start = match.start()
#         end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
#         header = match.group("header").strip()
#         chunk_text = text[start:end].strip()

#         # ✅ 무조건 Document 생성
#         doc_obj = Document(
#             page_content=chunk_text,
#             metadata={
#                 "chunk_id": f"{gri_id}_{i+1:02}",
#                 "section": header,
#                 "source": gri_id
#             }
#         )
#         all_documents.append(doc_obj)


# print(f"\n✅ 총 1차 청크 수: {len(all_documents)}")


# # ✅ 각 1차 청크(Document)를 1000자 단위로 분할
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.schema import Document

# def split_into_chunks(documents, max_chars=1000):
#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=max_chars,
#         chunk_overlap=20,
#         separators=["\n\n", "\n", ".", " ", ""]
#     )

#     new_docs = []
#     for doc in documents:
#         chunks = splitter.split_text(doc.page_content)
#         for i, chunk in enumerate(chunks):
#             new_docs.append(Document(
#                 page_content=chunk,
#                 metadata={
#                     "chunk_id": f"{doc.metadata['chunk_id']}_{i+1:02}",
#                     "source": doc.metadata["chunk_id"]
#                 }
#             ))
    
#     return new_docs


# # ✅ 2차 청크 생성 및 저장
# second_level_docs = split_into_chunks(all_documents)
# print(f"✅ 2차 청크 수: {len(second_level_docs)}")


In [15]:
# qa_results = []

# for doc in second_level_docs :  
#     context = doc.page_content
#     chunk_id = doc.metadata["chunk_id"]
#     source = doc.metadata["source"]

#     # 1. 질문 생성
#     question = llm.invoke(question_prompt.format(context=context)).content.strip()

#     # 2. 답변 생성
#     answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

#     # 3. 저장
#     qa_results.append({
#         "chunk_id": chunk_id,
#         "source": source,
#         "question": question,
#         "answer": answer,
#         "context": context[:300] + "..."  # 긴 context는 요약 출력
#     })


# print(len(qa_results))
# print(qa_results)

In [16]:
# #저장하기 json
# import json

# # 파일로 저장
# with open("GRI_qa_results.json", "w", encoding="utf-8") as f:
#     json.dump(qa_results, f, ensure_ascii=False, indent=2)

# print("✅ GRI_qa_results.json 파일 저장 완료")


# # 저장하기 vector
# from langchain.schema import Document

# qa_documents = []

# for item in qa_results:
#     qa_documents.append(Document(
#         page_content = f"Q: {item['question']}\nA: {item['answer']}",
#         metadata = {
#             "question": item["question"],
#             "chunk_id": item["chunk_id"],
#             "source": item["source"]
#         }
#     ))


# from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
# import key

# # 임베딩 모델 설정
# embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# # 벡터스토어로 저장
# qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
# qa_vectorstore.save_local("vectorstoresqa/GRI_qa_results_openai")

# print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/GRI_qa_results_openai")



In [17]:
# #평가하기
# from langchain.evaluation import load_evaluator
# from langchain_openai import ChatOpenAI
# import key

# # ✅ 평가에 사용할 LLM
# llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])




# # ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
# evaluator = load_evaluator("context_qa", llm=llm)

# # ✅ 평가 실행
# evaluated_results = []

# for item in qa_results:
#     result = evaluator.evaluate_strings(
#         input={"question": item["question"], "context": item["context"]},
#         prediction=item["answer"],
#         reference=item["answer"]  # ⬅️ 추가됨
#     )
#     evaluated_results.append({
#         "chunk_id": item["chunk_id"],
#         "question": item["question"],
#         "answer": item["answer"],
#         "context": item["context"],
#         "score": result["score"],
#         "reasoning": result.get("reasoning", "")
#     })


# # 결과 출력(상위 5개)
# import pandas as pd

# df_eval = pd.DataFrame(evaluated_results)
# display(df_eval[["chunk_id", "score", "question", "answer"]].head())


# #점수 구체화
# evaluator = load_evaluator(
#     evaluator="score_string",
#     criteria={"relevance": "질문과 답변의 관련성"},
#     normalize_by=10,
#     llm=llm
# )

# # DataFrame으로 변환
# df_eval = pd.DataFrame(evaluated_results)

# # CSV로 저장
# df_eval.to_csv("GRI_qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

# print("✅ LLM 평가 결과 저장 완료: GRI_qa_llm_eval_results.csv")



In [18]:
# # 코사인 유사도 검사
# from langchain_openai import OpenAIEmbeddings
# import key

# embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


# import numpy as np
# from numpy.linalg import norm

# def cosine_similarity(vec1, vec2):
#     return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


# evaluated_results = []

# for item in qa_results:
#     try:
#         answer_emb = embedding_model.embed_query(item["answer"])
#         context_emb = embedding_model.embed_query(item["context"])
#         score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
#     except Exception as e:
#         print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
#         score = None
    
#     evaluated_results.append({
#         "chunk_id": item["chunk_id"],
#         "question": item["question"],
#         "answer": item["answer"],
#         "score_cosine": score
#     })

# import pandas as pd

# df_eval = pd.DataFrame(evaluated_results)
# df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

# display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


# import pandas as pd

# # 1. DataFrame으로 변환
# df = pd.DataFrame(evaluated_results)

# # 2. CSV로 저장
# df.to_csv("GRI_qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

# print("✅ CSV 파일 저장 완료: GRI_qa_results_cosine_eval.csv")



In [19]:
#TCFD_implementing_Guidance
import fitz
import json
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

# 1. PDF 로드 및 섹션 정의
pdf_path = r"D:\TECHLAB\2차프로젝트_Chatbot\data\raw\TCFD-Implementing_Guidance.pdf"
doc = fitz.open(pdf_path)

section_page_defs = [
    ("chunk_01_Introduction", 4, 13),
    ("chunk_02_Recommendations", 13, 17),
    ("chunk_03_Guidance for All Sectors", 17, 24),
    ("chunk_04_financial", 24, 56),
    ("chunk_05_Supplemental Guidance for the Financial Sector", 56, 70),
    ("chunk_06_Fundamental Principles for Effective Disclosure", 70, 74),
    ("chunk_07_Appendix 1: Climate-Related Risks, Opportunities, and Financial Impacts", 74, 79),
    ("chunk_08_Appendix 2: Cross-Industry, Climate-Related Metric Categories", 79, 82),
    ("chunk_09_Appendix 3: Glossary and Abbreviations", 82, 85),
    ("chunk_10_Appendix 4: References", 85, 88),
]

# 2. 1차 섹션 청크 생성 (텍스트 + metadata)
documents1 = []
for chunk_id, start, end in section_page_defs:
    text = "\n".join(doc[i].get_text() for i in range(start, end)).strip()
    documents1.append(
        Document(page_content=text, metadata={"chunk_id": chunk_id, "source": "TCFD-Implementing_Guidance"})
    )



#TCFD_metrics_targets_guidance
import fitz  # PyMuPDF
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

# 1. PDF 열기
pdf_path = r"D:\TECHLAB\2차프로젝트_Chatbot\data\raw\TCFD-Metrics_Targets_Guidance.pdf"
doc = fitz.open(pdf_path)

# 2. 섹션 정의 (0-indexed page 기준)
section_page_defs = [
    ("chunk_01_Overview and Background", 1, 6),
    ("chunk_02_Scope and Approach", 6, 10),
    ("chunk_03_Climate-Related Metrics", 10, 29),
    ("chunk_04_Climate-Related Targets", 29, 38),
    ("chunk_05_Transition Plans", 38, 45),
    ("chunk_06_Financial Impacts", 45, 54),
    ("chunk_07_Appendix 1: Further Information on Select Cross-Industry, Climate-Related Metric Categories", 54, 61),
    ("chunk_08_Appendix 2: Example Disclosures", 61, 65),
    ("chunk_09_Appendix 3: Glossary and Abbreviations", 65, 68),
    ("chunk_10_Appendix 4: References", 68, 79),
]

# 3. 각 섹션을 Document로 변환
documents2 = []
for chunk_id, start, end in section_page_defs:
    text = "\n".join(doc[i].get_text() for i in range(start, end)).strip()
    documents2.append(Document(page_content=text, metadata={"chunk_id": chunk_id, "source": "TCFD-Metrics_Targets_Guidance"}))




# TCFD_report
import fitz  # PyMuPDF
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

# 1. PDF 열기
pdf_path = r"D:\TECHLAB\2차프로젝트_Chatbot\data\raw\TCFD-Report.pdf"
doc = fitz.open(pdf_path)

# 2. Section page range 정의 (0-indexed 기준, 실제 목차는 6페이지까지)
section_page_defs = [
    ("chunk_01_intro", 6, 7),
    ("chunk_02_risks", 7, 13),
    ("chunk_03_guidance", 13, 25),
    ("chunk_04_scenario", 25, 32),
    ("chunk_05_issues", 32, 41),
    ("chunk_06_conclusion", 41, 44),
    ("chunk_07_appendix_1", 44, 46),
    ("chunk_08_appendix_2", 46, 51),
    ("chunk_09_appendix_3", 51, 54),
    ("chunk_10_appendix_4", 54, 62),
    ("chunk_11_appendix_5", 62, 65),
    ("chunk_12_appendix_6", 65, 70),  # 끝
]

# 3. 각 section을 Document로 만들기
documents3 = []
for chunk_id, start, end in section_page_defs:
    text = "\n".join(doc[i].get_text() for i in range(start, end)).strip()
    documents3.append(Document(page_content=text, metadata={"chunk_id": chunk_id, "source": "TCFD-Report"}))


# ✅ 1차 전체 병합
all_documents = documents1 + documents2 + documents3
print(f"✅ 전체 1차 청크 수: {len(all_documents)}")


# ✅ 2차 청크 분할 함수 (1000자 기준)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_into_chunks(documents, max_chars=1000):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=20,
        separators=["\n\n", "\n", ".", " ", ""]
    )

    new_docs = []
    for doc in documents:
        chunks = splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            new_docs.append(Document(
                page_content=chunk,
                metadata={
                    "chunk_id": f"{doc.metadata['chunk_id']}_{i+1:02}",
                    "source": doc.metadata["chunk_id"]
                }
            ))
    
    return new_docs

# ✅ 2차 청크 생성 및 저장
second_level_docs = split_into_chunks(all_documents)
print(f"✅ 2차 청크 수: {len(second_level_docs)}")

✅ 전체 1차 청크 수: 32
✅ 2차 청크 수: 895


In [20]:
qa_results = []

for doc in second_level_docs :  
    context = doc.page_content
    chunk_id = doc.metadata["chunk_id"]
    source = doc.metadata["source"]

    # 1. 질문 생성
    question = llm.invoke(question_prompt.format(context=context)).content.strip()

    # 2. 답변 생성
    answer = llm.invoke(answer_prompt.format(context=context, question=question)).content.strip()

    # 3. 저장
    qa_results.append({
        "chunk_id": chunk_id,
        "source": source,
        "question": question,
        "answer": answer,
        "context": context[:300] + "..."  # 긴 context는 요약 출력
    })


print(len(qa_results))
print(qa_results)

895
[{'chunk_id': 'chunk_01_Introduction_01', 'source': 'chunk_01_Introduction', 'question': 'What additional materials has the Task Force on Climate-related Financial Disclosures issued to support organizations in implementing the recommendations and disclosing decision-useful climate-related financial information?', 'answer': 'The Task Force on Climate-related Financial Disclosures has issued other materials on specific topics intended to support organizations in implementing the recommendations and disclosing decision-useful climate-related financial information.', 'context': 'Implementing the Recommendations of the Task Force on Climate-related Financial Disclosures \n \n4\nA. \nIntroduction \n \nB. \nRecommendations \n \nC. \nGuidance for All Sectors \n \nD. \nSupplemental Guidance \nfor the Financial Sector \n \nE. \nSupplemental Guidance \nfor Non-Financial Groups \n \nF. \nFundamental Pr...'}, {'chunk_id': 'chunk_01_Introduction_02', 'source': 'chunk_01_Introduction', 'question

In [21]:
#저장하기 json
import json

# 파일로 저장
with open("TCFD_qa_results.json", "w", encoding="utf-8") as f:
    json.dump(qa_results, f, ensure_ascii=False, indent=2)

print("✅ TCFD_qa_results.json 파일 저장 완료")


# 저장하기 vector
from langchain.schema import Document

qa_documents = []

for item in qa_results:
    qa_documents.append(Document(
        page_content = f"Q: {item['question']}\nA: {item['answer']}",
        metadata = {
            "question": item["question"],
            "chunk_id": item["chunk_id"],
            "source": item["source"]
        }
    ))


from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import key

# 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])

# 벡터스토어로 저장
qa_vectorstore = FAISS.from_documents(qa_documents, embedding_model)
qa_vectorstore.save_local("vectorstoresqa/TCFD_qa_results_openai")

print("✅ QA 결과 벡터 저장 완료: vectorstoresqa/TCFD_qa_results_openai")



✅ TCFD_qa_results.json 파일 저장 완료
✅ QA 결과 벡터 저장 완료: vectorstoresqa/TCFD_qa_results_openai


In [22]:
#평가하기
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI
import key

# ✅ 평가에 사용할 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0, openai_api_key=key.key["OPEN_API_KEY"])




# ✅ evaluator 로딩 (질문, context, 답변 기준 평가)
evaluator = load_evaluator("context_qa", llm=llm)

# ✅ 평가 실행
evaluated_results = []

for item in qa_results:
    result = evaluator.evaluate_strings(
        input={"question": item["question"], "context": item["context"]},
        prediction=item["answer"],
        reference=item["answer"]  # ⬅️ 추가됨
    )
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "context": item["context"],
        "score": result["score"],
        "reasoning": result.get("reasoning", "")
    })


# 결과 출력(상위 5개)
import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
display(df_eval[["chunk_id", "score", "question", "answer"]].head())


#점수 구체화
evaluator = load_evaluator(
    evaluator="score_string",
    criteria={"relevance": "질문과 답변의 관련성"},
    normalize_by=10,
    llm=llm
)

# DataFrame으로 변환
df_eval = pd.DataFrame(evaluated_results)

# CSV로 저장
df_eval.to_csv("TCFD_qa_llm_eval_results.csv", index=False, encoding="utf-8-sig")

print("✅ LLM 평가 결과 저장 완료: TCFD_qa_llm_eval_results.csv")



,chunk_id,score,question,answer
0,chunk_01_Introduction_01,1,What additional materials has the Task Force o...,The Task Force on Climate-related Financial Di...
1,chunk_01_Introduction_02,1,What are the four overarching recommendations ...,The four overarching recommendations of the Ta...
2,chunk_01_Introduction_03,1,What changes were made in the supplemental gui...,The changes made in the supplemental guidance ...
3,chunk_01_Introduction_04,1,What specific guidance has been revised in the...,The specific guidance that has been revised in...
4,chunk_01_Introduction_05,1,은행들이 탄소 관련 자산에 대한 신용 위험 집중도 정보를 공개할 때 어떤 섹터의 자...,에너지 및 공공서비스 섹터에 연결된 자산을 사용하는 것을 제안했습니다.


This chain was only tested with GPT-4. Performance may be significantly worse with other models.


✅ LLM 평가 결과 저장 완료: TCFD_qa_llm_eval_results.csv


In [23]:
# 코사인 유사도 검사
from langchain_openai import OpenAIEmbeddings
import key

embedding_model = OpenAIEmbeddings(openai_api_key=key.key["OPEN_API_KEY"])


import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


evaluated_results = []

for item in qa_results:
    try:
        answer_emb = embedding_model.embed_query(item["answer"])
        context_emb = embedding_model.embed_query(item["context"])
        score = cosine_similarity(np.array(answer_emb), np.array(context_emb))
    except Exception as e:
        print(f"❌ 오류 발생 (chunk_id: {item['chunk_id']}): {e}")
        score = None
    
    evaluated_results.append({
        "chunk_id": item["chunk_id"],
        "question": item["question"],
        "answer": item["answer"],
        "score_cosine": score
    })

import pandas as pd

df_eval = pd.DataFrame(evaluated_results)
df_sorted = df_eval.sort_values(by="score_cosine", ascending=False)

display(df_sorted[["chunk_id", "score_cosine", "question"]].head(10))


import pandas as pd

# 1. DataFrame으로 변환
df = pd.DataFrame(evaluated_results)

# 2. CSV로 저장
df.to_csv("TCFD_qa_results_cosine_eval.csv", index=False, encoding="utf-8-sig")

print("✅ CSV 파일 저장 완료: TCFD_qa_results_cosine_eval.csv")



,chunk_id,score_cosine,question
412,chunk_03_Climate-Related Metrics_39,0.986782,What were some of the questions asked to users...
796,chunk_06_conclusion_06,0.986451,What does Task Force appreciate regarding clim...
154,chunk_04_financial_78,0.978322,How should organizations describe their proces...
794,chunk_06_conclusion_04,0.977243,Why did the Task Force choose a one billion US...
188,chunk_04_financial_112,0.974842,What should organizations consider when provid...
677,chunk_02_risks_19,0.974195,What are the two major categories of climate-r...
627,chunk_10_Appendix 4: References_17,0.972762,What is the title of the document published by...
801,chunk_06_conclusion_11,0.972458,What were some common themes in the feedback p...
179,chunk_04_financial_103,0.972203,Why might organizations agree to meet investor...
240,chunk_05_Supplemental Guidance for the Financi...,0.971501,What does the Task Force suggest financial org...


✅ CSV 파일 저장 완료: TCFD_qa_results_cosine_eval.csv
